In [ ]:
import mne
import numpy as np
import os

# List of EEG files
eeg_files = {
    "pizza": "pizza.set",
    "apple": "apple.set",
    "hammer": "hammer.set"
}

# Directory where EEG files are located
data_dir = "path_to_your_data"

# Parameters for filtering and preprocessing
high_pass = 0.5  # Hz
low_pass = 30  # Hz
notch_freq = 50  # Hz (UK powerline noise)
target_sfreq = 500  # Target sampling rate
n_channels = 32  # Number of EEG channels

# Function to preprocess EEG data
def preprocess_eeg(file_path):
    # Load EEG data
    raw = mne.io.read_raw_eeglab(file_path, preload=True)

    # Ensure correct channel naming
    if len(raw.ch_names) != n_channels:
        print(f"Warning: Expected {n_channels} channels, but found {len(raw.ch_names)}.")

    # Resample data if the sampling rate is different from 500 Hz
    if raw.info['sfreq'] != target_sfreq:
        print(f"Resampling from {raw.info['sfreq']} Hz to {target_sfreq} Hz...")
        raw.resample(target_sfreq)

    # Apply band-pass filtering (0.5 Hz - 30 Hz)
    raw.filter(l_freq=high_pass, h_freq=low_pass, fir_design='firwin')

    # Apply notch filter to remove 50 Hz powerline noise
    raw.notch_filter(freqs=notch_freq, fir_design='firwin')

    # Set EEG reference to the average reference
    raw.set_eeg_reference('average')

    # Apply ICA to remove eye-blink artifacts
    ica = mne.preprocessing.ICA(n_components=32, random_state=97, max_iter=500)
    ica.fit(raw)

    # Automatically detect eye-blink artifacts
    eog_indices, _ = ica.find_bads_eog(raw, threshold=3.0)

    # Remove eye-blink related components
    ica.exclude = eog_indices
    raw_clean = ica.apply(raw)

    return raw_clean

# Process each EEG file
cleaned_data = {}

for label, file_name in eeg_files.items():
    file_path = os.path.join(data_dir, file_name)
    print(f"Processing {label} data...")
    cleaned_data[label] = preprocess_eeg(file_path)

# Save cleaned data
for label, raw_clean in cleaned_data.items():
    output_file = os.path.join(data_dir, f"{label}_clean.fif")
    raw_clean.save(output_file, overwrite=True)
    print(f"Saved cleaned EEG data for {label} as {output_file}")

print("EEG Preprocessing Completed.")
